In [4]:
import pandas as pd

from collections import Counter

import re

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

from nrclex import NRCLex

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.stem import PorterStemmer

import spacy

from langdetect import detect

#import nlp

import langid

import collections

from langdetect.lang_detect_exception import LangDetectException

import locationtagger

from wordcloud import WordCloud

import gensim

from gensim.utils import simple_preprocess

import nltk

from nltk.corpus import stopwords

from gensim.models.ldamulticore import LdaMulticore

import gensim.corpora as corpora

import gensim.corpora as MmCorpus

from gensim import models

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

import matplotlib.pyplot as plt

from googletrans import Translator

In [5]:
df = pd.read_csv('/Users/beltran/Desktop/influence_marketing/influence_marketing_reco/datasets/df_dinal.csv')

In [6]:
def preprocess_text(text):

    stop_words = ['english', 'spanish', 'portuguese', 'french', 'arabic', 'indonesian', 'russian', 'italian']
    
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words(stop_words)]

    stemmer=PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

In [7]:
df['user_Description'] = df['user_Description'].fillna('')
df['post_Description'] = df['post_Description'].fillna('')

In [ ]:
# apply the function df
df['user_Description'] = df['user_Description'].apply(preprocess_text)
df['post_Description'] = df['post_Description'].apply(preprocess_text)
df

In [ ]:
df['all_descriptions'] = df['user_Description'].str.cat(df['post_Description'], sep=' ')

In [ ]:
def detect_languages(text):
    if len(text) < 10:
        return []
    try:
        lang = detect(text)
        return [lang]
    except LangDetectException:
        return []


languageslist = df["all_descriptions"].apply(detect_languages)
print(languageslist)


0        [en]
1        [en]
2        [es]
3        [en]
4        [en]
         ... 
23281    [en]
23282    [en]
23283    [en]
23284    [en]
23285    [en]
Name: all_descriptions, Length: 23286, dtype: object


In [ ]:
import spacy

models = spacy.util.get_installed_models()

print("The following models are installed:")
for model in models:
    print(model)

The following models are installed:
es_core_news_lg
en_core_web_lg
fr_core_news_lg
it_core_news_lg
xx_ent_wiki_sm
pt_core_news_lg
en_core_web_sm
es_core_news_sm


In [ ]:
nlp_english = spacy.load("en_core_web_lg")
nlp_spanish = spacy.load("es_core_news_lg")
nlp_portuguese = spacy.load("pt_core_news_lg")
nlp_french = spacy.load("fr_core_news_lg")
nlp_italian = spacy.load("it_core_news_lg")
nlp_any = spacy.load("xx_ent_wiki_sm")


def extract_locations(text):
    if len(text) < 10:  # Check if text is at least 10 characters long
        return []
    try:
        lang_code = detect(text)
    except LangDetectException:
        return []
    
    if lang_code == "en":
        nlp = nlp_english
    elif lang_code == "es":
        nlp = nlp_spanish
    elif lang_code == "pt":
        nlp = nlp_portuguese
    elif lang_code == "it":
        nlp = nlp_italian
    elif lang_code == "fr":
        nlp = nlp_french
    elif lang_code in ['ar', 'bg', 'ca', 'cs', 'da', 'de', 'el', 'en', 'es', 'et', 'fa', 'fi', 'fr', 'he', 'hi', 'hr', 'hu', 'id', 'it', 'ja', 'kk', 'ko', 'lt', 'lv', 'mk', 'ml', 'mr', 'nb', 'nl', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'sq', 'sr', 'sv', 'ta', 'th', 'tr', 'uk', 'ur', 'vi', 'zh']:
        nlp = nlp_any
    else:
        return []
    locations = locationtagger.find_locations(text = text)
    locations = locations.countries #locations.country_regions, locations.country_cities, locations.other_countries, locations.region_cities, locations.other_regions, locations.other
    return locations

df['locations2'] = df["all_descriptions"].apply(extract_locations)
df

,user_Name,followers,user_Description,post_Url,likes,post_Description,post_Location,ID,location,hashtags,all_descriptions,locations2
0,chiquis,"5,721,916",chiqui dio ! -bff 4life ! singer entrepreneur ...,https://www.instagram.com/p/CqHKmO-vNQu/,NaN,true stori . 💜💫 @ beflawlessskin make fall lov...,NaN,416,NaN,[],chiqui dio ! -bff 4life ! singer entrepreneur ...,[]
1,chiquis,"5,721,916",chiqui dio ! -bff 4life ! singer entrepreneur ...,https://www.instagram.com/p/CqB1sDtN2gG/,NaN,take note “ quier pued ” mindset 😍💫 # womenshi...,NaN,416,NaN,['womenshistorymonth'],chiqui dio ! -bff 4life ! singer entrepreneur ...,[]
2,chiquis,"5,721,916",chiqui dio ! -bff 4life ! singer entrepreneur ...,https://www.instagram.com/p/CqBkI9MNZhR/,NaN,gracia mexico ! ! ! ! ! ❤️‍🔥😭🙌🏻 @ monitorlatin...,NaN,416,NaN,"['monitorlatino', 'chiquis', 'porquesoyabejare...",chiqui dio ! -bff 4life ! singer entrepreneur ...,[mexico]
3,chiquis,"5,721,916",chiqui dio ! -bff 4life ! singer entrepreneur ...,https://www.instagram.com/p/CqBSU7ev5bz/,NaN,’ talk enough latin community—ivf surrogaci . ...,NaN,416,NaN,['chiquisandchill'],chiqui dio ! -bff 4life ! singer entrepreneur ...,[]
4,chiquis,"5,721,916",chiqui dio ! -bff 4life ! singer entrepreneur ...,https://www.instagram.com/p/CqBN0PYJpgE/,NaN,’ honor celebr # womenshistorymonth @ pandora ...,NaN,416,NaN,"['womenshistorymonth', 'pandoramusic', 'pandor...",chiqui dio ! -bff 4life ! singer entrepreneur ...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
23281,wisdm,NaN,"wisdom kay director , stylist , photograph , e...",https://www.instagram.com/p/CqOMASSOM8i/,botshelllloo,world collid # puma x # finalfantasyxiv # hype...,NaN,2156,NaN,"['puma', 'finalfantasyxiv', 'hypebeast', 'ad']","wisdom kay director , stylist , photograph , e...",[]
23282,wisdm,NaN,"wisdom kay director , stylist , photograph , e...",https://www.instagram.com/p/CqB2J9oPmJB/,734.601 Me gusta,high fashion phinea ferb 👀 alot fun style char...,NaN,2156,NaN,[],"wisdom kay director , stylist , photograph , e...",[]
23283,wisdm,NaN,"wisdom kay director , stylist , photograph , e...",https://www.instagram.com/p/Cp_RDoqMKPg/,lyricreneee,’ go post jacket like 20 time,NaN,2156,NaN,[],"wisdom kay director , stylist , photograph , e...",[]
23284,wisdm,NaN,"wisdom kay director , stylist , photograph , e...",https://www.instagram.com/p/Cp8QxZ7Ovjv/,pedrozeo,sane fashion enjoy :,NaN,2156,NaN,[],"wisdom kay director , stylist , photograph , e...",[]
